In [273]:
import random
import numpy as np
from scipy import integrate

Z = 2
e = (1.602e-19 * np.sqrt(1/(4*np.pi*8.854e-12))) # J
e_sq_raw = e**2
e_sq = (e**2 / 1.6e-19) * 1e10 #14.4 # eV A
hbar_raw = (6.63e-34 / (2*np.pi))
hbar = hbar_raw * 1e10 # A
pi = np.pi
m_e_raw = 9.11e-31 # kg 
m_e = m_e_raw * 1e10 # kg A
r0 = (hbar_raw**2 / (e_sq_raw*m_e_raw)) * 1e10 # 0.5297 # A, hbar**2/m_e*e**2

upper = 3

class VMC():
    def __init__(self,r1,r2,j,k,m,chi=1,Z=Z,r0=r0,bounds=[0,upper],num_samples=10):
        self.norm = self.find_norm(j,k,m,chi,bounds)
        self.MC_importance_sampling(self.local_energy,self.probability_density,j,k,m,num_samples,bounds,chi)
        
    def test_function(self,r1,r2,j,k,m,chi=1,Z=Z,r0=r0):
        return (r1+r2)**j * (r1-r2)**k * np.exp(-(Z/(r0*chi))*(r1+r2))

    def mod_sq_function(self,r1,r2,j,k,m,chi=1,Z=Z,r0=r0):
        return (r1+r2)**(2*j) * (r1-r2)**(2*k) * np.exp(-(2*Z/(r0*chi))*(r1+r2))

    def transformation_method(self,function,num_samples,bounds,j,k,m,chi,Z,r0): # BOUNDS ERRONEOUS - function call wrong!
        sample_r1 = np.arange(bounds[0],bounds[1],(bounds[1]-bounds[0])/100)
        sample_r2 = np.arange(bounds[0],bounds[1],(bounds[1]-bounds[0])/100)
        print([function(mod_sq_func,r1,r2,j,k,m,chi,Z,r0) for el_r1,el_r2 in zip(sample_r1,sample_r2)])
        m = max([function(mod_sq_func,r1,r2,j,k,m,chi,Z,r0) for el_r1,el_r2 in zip(sample_r1,sample_r2)])
        print('\nMaximum distribution value: {:.2f}'.format(m))
        count,total = 0,0
        sample = []
        while count < num_samples:
            total += 1
            r = random.random()
            r1 = bounds[0] + (bounds[1] - bounds[0]) * random.random() # CHANGE FROM UNIFORM SAMPLING?
            r2 = bounds[0] + (bounds[1] - bounds[0]) * random.random()
            if r * m < function(mod_sq_func,r1,r2,j,k,m,chi,Z,r0):
                count += 1
                sample.append([r1,r2,function(mod_sq_func,r1,r2,j,k,m,chi,Z,r0)])
        hit = count/total
        print('\nPercentage hit samples: {:.2f}%'.format(hit))
        return np.array(sample)

    def find_norm(self,j,k,m,chi,bounds):
        integrand = self.mod_sq_function
        def r1int(r2,j,k,m,int_max,chi):
            return integrate.quad(integrand,0,int_max,args=(r2,j,k,m,chi))[0]
        norm,err = integrate.quad(lambda r2: r1int(r2,j,k,m,bounds[1],chi),bounds[0],bounds[1])
        return norm
    
    def probability_density(self,mod_sq_func,r1,r2,j,k,m,chi,Z,r0):
        return (1/self.norm) * mod_sq_func(r1,r2,j,k,m,chi,Z,r0)

    def acting_hamiltonian(self,r1,r2,j,k,m,chi=1): # change for every structurally different test function, find analytically
        return #...

    def local_energy(self,acting_hamiltonian,test_function,r1,r2,j=0,k=0,m=0,chi=1):
        return acting_hamiltonian(r1,r2,j,k,m,chi) / test_function(r1,r2,j,k,m,chi)

    def MC_importance_sampling(self,operator_func,dist_func,j,k,m,num_samples,bounds,chi=1):
        r1s,r2s = transformation_method(dist_func,num_samples,bounds,j,k,m,chi).T[:2]
        return r1s,r2s #... finish off, this is just a test
    
    def __call__(self):
        return #...

import matplotlib.pyplot as plt


VMC(1,1,1,1,1)
'''
r1s,r2s = MC_importance_sampling(local_energy,probability_density,j=1,k=1,m=0,num_samples=200,chi=1)
plt.plot(np.arange(0,3,0.01),probability_density(np.arange(0,3,0.01),r2=0,j=1,k=2,m=0,chi=1,integral_max=3))
plt.hist(r1s,density=True)
plt.show()
plt.plot(np.arange(0,3,0.01),probability_density(np.arange(0,3,0.01),r2=0,j=1,k=2,m=0,chi=1,integral_max=3))
plt.hist(r2s,density=True)
plt.show()'''

0.00021614184534715912


TypeError: probability_density() missing 2 required positional arguments: 'Z' and 'r0'